<a href="https://colab.research.google.com/github/kshitij9969/Indicator_of_sucess_dataset/blob/main/Indicator_of_sucess_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Cost of living indicator

### Cost of living indicator over the years. 

In [ ]:
soup = BeautifulSoup(res, 'html.parser')